**DeapSECURE module 5: Cryptography for Privacy-Preserving Computation (part A: Data Protection)**

# Session 2 Solution: AES Encryption and Decryption

This notebook provides complete solutions for AES encryption and decryption exercises,
including practical examples with both educational and production-grade implementations.

## 1. Setup and Helper Functions

In [1]:
!pip install aes

In [2]:
import time
import numpy as np

In [3]:
def encode_int(C, minlength=16):
    """Encodes an arbitrarily long integer into a bytes object."""
    if isinstance(C, list):
        # Handle list output from aes module
        C = bytes(C)
        if len(C) < minlength:
            C = C.rjust(minlength, b'\x00')
        return C
    C_hex = hex(C)[2:]
    if len(C_hex) % 2:
        C_hex = '0' + C_hex
    C_bytes = bytes.fromhex(C_hex)
    if len(C_bytes) < minlength:
        C_bytes = C_bytes.rjust(minlength, b'\x00')
    return C_bytes

def decode_int(B):
    """Decodes a bytes object into a long integer."""
    return int(B.hex(), 16)

def leftpad16(B):
    """Pad a bytes array from the left with NULL chars to multiple of 16 bytes."""
    padlength = len(B) % 16
    if padlength > 0:
        return (b'\x00' * (16 - padlength)) + B
    else:
        return B

def aes_list_to_int(aes_output):
    """Convert aes module list output to integer."""
    if isinstance(aes_output, list):
        return int.from_bytes(bytes(aes_output), 'big')
    return aes_output

## 2. Educational AES Implementation (aes.py)

The aes module provides a simple, pure-Python implementation for learning purposes.

In [4]:
# Import the educational AES module
import aes

### 2.1 Key Generation and Setup

In [5]:
# Define a secret key (must be less than 128 bits / 16 bytes)
secret_key = 0x5e413c
print(f"Secret key (hex): {hex(secret_key)}")
print(f"Secret key (decimal): {secret_key}")

# Create AES encryptor/decryptor object
E = aes.aes(secret_key)
print(f"\nAES object created successfully")

Secret key (hex): 0x5e413c
Secret key (decimal): 6177084

AES object created successfully


### 2.2 Plaintext Definition

In [6]:
# Define plaintext (must be under 16 bytes)
plaintext_string = 'IdeaFusion'
plaintext = plaintext_string.encode()  # UTF-8 encoding
plaintext_int = decode_int(plaintext)

print(f"Plaintext string: '{plaintext_string}'")
print(f"Plaintext bytes: {plaintext}")
print(f"Plaintext (decimal): {plaintext_int}")
print(f"Plaintext (hex): {hex(plaintext_int)}")

Plaintext string: 'IdeaFusion'
Plaintext bytes: b'IdeaFusion'
Plaintext (decimal): 346584732854395888824174
Plaintext (hex): 0x49646561467573696f6e


### 2.3 Encryption

In [7]:
# Encrypt the plaintext
ciphertext_list = E.enc_once(plaintext_int)
ciphertext_int = aes_list_to_int(ciphertext_list)

print(f"Ciphertext (decimal): {ciphertext_int}")
print(f"Ciphertext (hex): {hex(ciphertext_int)}")

# Convert to bytes
ciphertext_bytes = encode_int(ciphertext_list)
print(f"Ciphertext (bytes): {ciphertext_bytes}")
print(f"Ciphertext (hex bytes): {ciphertext_bytes.hex()}")

Ciphertext (decimal): 24380479071008565588727530699715278575
Ciphertext (hex): 0x1257826ca28522eab4860da1088876ef
Ciphertext (bytes): b'\x12W\x82l\xa2\x85"\xea\xb4\x86\r\xa1\x08\x88v\xef'
Ciphertext (hex bytes): 1257826ca28522eab4860da1088876ef


In [8]:
# Analysis of ciphertext
print(f"Original message length: {len(plaintext)} bytes")
print(f"Encrypted message length: {len(ciphertext_bytes)} bytes")
print(f"\nNote: Both are 16 bytes because AES operates on 16-byte blocks")
print(f"The ciphertext appears as random bytes (not readable)")

Original message length: 10 bytes
Encrypted message length: 16 bytes

Note: Both are 16 bytes because AES operates on 16-byte blocks
The ciphertext appears as random bytes (not readable)


### 2.4 Decryption with Correct Key

In [9]:
# Decrypt with the correct key
decrypted_list = E.dec_once(ciphertext_int)
decrypted_int = aes_list_to_int(decrypted_list)
decrypted_bytes = encode_int(decrypted_list)

print(f"Decrypted (hex): {hex(decrypted_int)}")
print(f"Decrypted (bytes): {decrypted_bytes}")

# Remove padding (leading NULLs)
decrypted_message = decrypted_bytes.lstrip(b'\x00')
print(f"\nDecrypted message: {decrypted_message}")
print(f"Matches original: {decrypted_message == plaintext}")

Decrypted (hex): 0x49646561467573696f6e
Decrypted (bytes): b'\x00\x00\x00\x00\x00\x00IdeaFusion'

Decrypted message: b'IdeaFusion'
Matches original: True


### 2.5 Decryption with Wrong Key

In [10]:
# Create AES object with wrong key
wrong_key = 0x27F6A123
E_wrong = aes.aes(wrong_key)

# Try to decrypt with wrong key
wrong_decrypted_list = E_wrong.dec_once(ciphertext_int)
wrong_decrypted_bytes = encode_int(wrong_decrypted_list)
wrong_message = wrong_decrypted_bytes.lstrip(b'\x00')

print(f"Decrypted with wrong key: {wrong_message}")
print(f"Matches original: {wrong_message == plaintext}")
print(f"\nConclusion: Without the correct key, decryption produces garbage!")

Decrypted with wrong key: b"/\xf3/\xbf\r*\xddw\xdc\x05\x9b*\x1b'\xa0\x05"
Matches original: False

Conclusion: Without the correct key, decryption produces garbage!


In [11]:
# Try multiple wrong keys
print("Attempting decryption with various wrong keys:")
for i in range(5):
    wrong_key_i = secret_key + i + 1
    E_i = aes.aes(wrong_key_i)
    decrypted_i = E_i.dec_once(ciphertext_int)
    msg_i = encode_int(decrypted_i).lstrip(b'\x00')
    print(f"Key {hex(wrong_key_i)}: {msg_i}")

Attempting decryption with various wrong keys:
Key 0x5e413d: b'\xdf\x8f\xea\x14q\xa8\xaa\xf5\xc6D!|\x11q\xa0\xe7'
Key 0x5e413e: b'\xa0\x94<\x84N\xb4\xf4Q\xe9\xbb\x8aC\xec\xecK\x8a'
Key 0x5e413f: b'\xdc\x99d\xe9\xfa\xc9\x07"\x19\xd4\x9c4\xef\xf0\xf0Y'
Key 0x5e4140: b'%\x1cJ\xdf\x19\x96H\x1c\xb7z\xf8\xc1\x02\xb0\xe5\xfa'
Key 0x5e4141: b'\xd2\x89\x8c\xaa;\xcd6Q\xc5d\x1e\x01\xc1P\xa4D'


### 2.6 Encryption with Different Keys

In [12]:
# Encrypt with similar keys
print(f"Original key: {hex(secret_key)}")
print(f"Original ciphertext: {hex(ciphertext_int)}\n")

for i in range(1, 4):
    key_i = secret_key + i
    E_i = aes.aes(key_i)
    cipher_i = aes_list_to_int(E_i.enc_once(plaintext_int))
    print(f"Key {hex(key_i)}: {hex(cipher_i)}")

print("\nObservation: Even slightly different keys produce completely different ciphertexts!")
print("This is the avalanche effect - a key property of secure encryption.")

Original key: 0x5e413c
Original ciphertext: 0x1257826ca28522eab4860da1088876ef

Key 0x5e413d: 0xd9e5a88ee44ab09fd5b3b31efd7ead4a
Key 0x5e413e: 0x18d11a99890bf629c0e4d45ceae6cc61
Key 0x5e413f: 0xc79dc6d42000d7f2facb1f84bca8a82d

Observation: Even slightly different keys produce completely different ciphertexts!
This is the avalanche effect - a key property of secure encryption.


### 2.7 Performance Analysis

In [13]:
# Measure encryption performance
print("Encryption Performance (aes.py - pure Python):")
%timeit -n 100 -r 5 E.enc_once(plaintext_int)

Encryption Performance (aes.py - pure Python):
59.5 μs ± 2.25 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)


In [14]:
# Measure decryption performance
print("Decryption Performance (aes.py - pure Python):")
%timeit -n 100 -r 5 E.dec_once(ciphertext_int)

Decryption Performance (aes.py - pure Python):
77.1 μs ± 1.08 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)


In [15]:
# Key cracking time estimation
print("Key Cracking Time Estimates:")
print("\nAssuming 0.2 seconds per 1000 encryptions (0.0002 sec/encryption):\n")

key_sizes = [8, 16, 24, 32]
for bits in key_sizes:
    combinations = 2 ** bits
    time_seconds = combinations * 0.0002
    time_hours = time_seconds / 3600
    time_days = time_hours / 24
    time_years = time_days / 365
    
    print(f"{bits}-bit key: {combinations:,} combinations")
    if time_seconds < 60:
        print(f"  Time: {time_seconds:.2f} seconds")
    elif time_hours < 24:
        print(f"  Time: {time_hours:.2f} hours")
    elif time_days < 365:
        print(f"  Time: {time_days:.2f} days")
    else:
        print(f"  Time: {time_years:.2f} years")
    print()

Key Cracking Time Estimates:

Assuming 0.2 seconds per 1000 encryptions (0.0002 sec/encryption):

8-bit key: 256 combinations
  Time: 0.05 seconds

16-bit key: 65,536 combinations
  Time: 13.11 seconds

24-bit key: 16,777,216 combinations
  Time: 0.93 hours

32-bit key: 4,294,967,296 combinations
  Time: 9.94 days



## 3. Production-Grade AES (PyCryptodome)

PyCryptodome provides optimized, production-ready cryptographic implementations.

In [16]:
# Install pycryptodome (provides Crypto module)
# Note: Uninstall conflicting 'crypto' package if needed
!pip uninstall -y crypto 2>/dev/null || true
!pip install --force-reinstall pycryptodome

  Using cached pycryptodome-3.23.0-cp37-abi3-macosx_10_9_universal2.whl.metadata (3.4 kB)
Using cached pycryptodome-3.23.0-cp37-abi3-macosx_10_9_universal2.whl (2.5 MB)
  Attempting uninstall: pycryptodome
    Found existing installation: pycryptodome 3.23.0
    Uninstalling pycryptodome-3.23.0:
      Successfully uninstalled pycryptodome-3.23.0


In [17]:
# Import PyCryptodome AES
from Crypto.Cipher import AES

### 3.1 ECB Mode (Educational - Not Recommended for Production)

In [18]:
# Create secret key in bytes format
secret_bkey = encode_int(secret_key)
print(f"Secret key (hex): {secret_bkey.hex()}")
print(f"Secret key (bytes): {secret_bkey}")

# Create AES cipher in ECB mode
EE = AES.new(secret_bkey, AES.MODE_ECB)
print(f"\nAES cipher created in ECB mode")

Secret key (hex): 000000000000000000000000005e413c
Secret key (bytes): b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00^A<'

AES cipher created in ECB mode


In [19]:
# Encrypt with PyCryptodome
plaintext_padded = leftpad16(b'IdeaFusion')
ciphertext_pycrypto = EE.encrypt(plaintext_padded)

print(f"Plaintext (padded): {plaintext_padded}")
print(f"Ciphertext: {ciphertext_pycrypto}")
print(f"Ciphertext (hex): {ciphertext_pycrypto.hex()}")

Plaintext (padded): b'\x00\x00\x00\x00\x00\x00IdeaFusion'
Ciphertext: b'\x12W\x82l\xa2\x85"\xea\xb4\x86\r\xa1\x08\x88v\xef'
Ciphertext (hex): 1257826ca28522eab4860da1088876ef


In [20]:
# Decrypt with PyCryptodome
decrypted_pycrypto = EE.decrypt(ciphertext_pycrypto)
print(f"Decrypted: {decrypted_pycrypto}")
print(f"Decrypted (unpadded): {decrypted_pycrypto.lstrip(b'\x00')}")

Decrypted: b'\x00\x00\x00\x00\x00\x00IdeaFusion'
Decrypted (unpadded): b'IdeaFusion'


In [21]:
# Encrypt longer message
long_message = b'The master key (a secret) must be less than 128 bits (16 bytes)'
long_padded = leftpad16(long_message)
print(f"Original length: {len(long_message)} bytes")
print(f"Padded length: {len(long_padded)} bytes")

long_ciphertext = EE.encrypt(long_padded)
print(f"Ciphertext length: {len(long_ciphertext)} bytes")

long_decrypted = EE.decrypt(long_ciphertext)
print(f"\nDecrypted matches original: {long_decrypted.lstrip(b'\x00') == long_message}")

Original length: 63 bytes
Padded length: 64 bytes
Ciphertext length: 64 bytes

Decrypted matches original: True


### 3.2 Performance Comparison

In [22]:
# Performance: aes.py (pure Python)
print("Pure Python AES (aes.py):")
%timeit -n 100 -r 5 E.enc_once(plaintext_int)

Pure Python AES (aes.py):
68.4 μs ± 14.8 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)


In [23]:
# Performance: PyCryptodome (C-optimized)
print("PyCryptodome AES (C-optimized):")
%timeit -n 1000 -r 5 EE.encrypt(plaintext_padded)

PyCryptodome AES (C-optimized):
986 ns ± 592 ns per loop (mean ± std. dev. of 5 runs, 1,000 loops each)


In [24]:
print("\nConclusion: PyCryptodome is ~30-100x faster than pure Python!")
print("This is because performance-critical operations are implemented in C.")


Conclusion: PyCryptodome is ~30-100x faster than pure Python!
This is because performance-critical operations are implemented in C.


### 3.3 EAX Mode (Authenticated Encryption - Recommended for Production)

In [25]:
# Create AES cipher in EAX mode (authenticated encryption)
EAX = AES.new(secret_bkey, AES.MODE_EAX)
EAX_nonce = EAX.nonce

print(f"EAX nonce: {EAX_nonce.hex()}")
print(f"Nonce length: {len(EAX_nonce)} bytes")

EAX nonce: 02b0dd2929e55e16b6a894c56359905d
Nonce length: 16 bytes


In [26]:
# Encrypt with EAX
message = leftpad16(b'The master key (a secret) must be less than 128 bits (16 bytes)')
ciphertext_eax = EAX.encrypt(message)

print(f"Ciphertext (EAX): {ciphertext_eax.hex()}")

Ciphertext (EAX): 319bbaebefe005e3dfbaad683d0845a4b2b70830944ff344225ffd13263bfdb87ea4e77b86b91082a12dbce4ddbbca66892af6c0a5ee5a202838cd0558713185


In [27]:
# Decrypt with EAX (need same nonce)
DD_EAX = AES.new(secret_bkey, AES.MODE_EAX, nonce=EAX_nonce)
decrypted_eax = DD_EAX.decrypt(ciphertext_eax)

print(f"Decrypted: {decrypted_eax.lstrip(b'\x00')}")

Decrypted: b'The master key (a secret) must be less than 128 bits (16 bytes)'


In [28]:
# Demonstrate that EAX produces different ciphertexts each time (due to random nonce)
print("EAX Mode - Different Ciphertexts Each Time (due to random nonce):\n")

for i in range(3):
    EAX_i = AES.new(secret_bkey, AES.MODE_EAX)
    cipher_i = EAX_i.encrypt(message)
    print(f"Attempt {i+1}: {cipher_i.hex()[:32]}...")

print("\nNote: Each encryption produces different ciphertext (more secure!)")

EAX Mode - Different Ciphertexts Each Time (due to random nonce):

Attempt 1: afa36d61d8cc1a07a9ac7f81a39a4046...
Attempt 2: 434284c0954547fb1abb6d9758060e25...
Attempt 3: 3729c29b9586d1c0455ee39677b13870...

Note: Each encryption produces different ciphertext (more secure!)


## 4. Key Cracking Challenge

Demonstrates the computational difficulty of brute-force key recovery.

In [29]:
# Create a challenge: encrypt a message with unknown key
challenge_key = 0xABCDEF  # Unknown to us
challenge_message = b'SECRET'

E_challenge = aes.aes(challenge_key)
challenge_plaintext_int = decode_int(challenge_message)
challenge_ciphertext_list = E_challenge.enc_once(challenge_plaintext_int)
challenge_ciphertext = aes_list_to_int(challenge_ciphertext_list)

print(f"Challenge ciphertext: {hex(challenge_ciphertext)}")
print(f"\nTask: Find the key that decrypts this message to a readable string")

Challenge ciphertext: 0xb91b29c5845ff40bc8e7e684b793145e

Task: Find the key that decrypts this message to a readable string


In [30]:
# Brute force attack on 16-bit key space
def is_valid_message(msg_bytes):
    """Check if decrypted message contains only printable ASCII characters."""
    msg = msg_bytes.lstrip(b'\x00')
    if len(msg) == 0:
        return False
    try:
        msg.decode('ascii')
        return all(32 <= b < 127 for b in msg)  # Printable ASCII
    except:
        return False

print("Attempting brute-force attack on 16-bit key space...\n")

start_time = time.time()
found = False

for key_attempt in range(0x10000):  # 16-bit key space
    E_attempt = aes.aes(key_attempt)
    decrypted = E_attempt.dec_once(challenge_ciphertext)
    decrypted_bytes = encode_int(decrypted)
    
    if is_valid_message(decrypted_bytes):
        msg = decrypted_bytes.lstrip(b'\x00')
        print(f"Found! Key: {hex(key_attempt)}")
        print(f"Message: {msg}")
        found = True
        break

end_time = time.time()
print(f"\nTime taken: {end_time - start_time:.2f} seconds")
if found:
    print(f"Correct key was: {hex(challenge_key)}")

Attempting brute-force attack on 16-bit key space...

Found! Key: 0xdc62
Message: b'R]WO C7GWgT`%ZQ{'

Time taken: 4.56 seconds
Correct key was: 0xabcdef


## 5. Summary: Key Takeaways

Important lessons from AES encryption and decryption.

In [31]:
print("="*70)
print("KEY TAKEAWAYS: AES ENCRYPTION AND DECRYPTION")
print("="*70)

print("""
1. ENCRYPTION BASICS:
   - AES operates on 16-byte blocks
   - Plaintext must be padded to multiple of 16 bytes
   - Same plaintext + same key = same ciphertext (in ECB mode)

2. KEY SECURITY:
   - Without the correct key, decryption produces garbage
   - Small key differences produce completely different ciphertexts
   - This is the "avalanche effect" - essential for security

3. PERFORMANCE:
   - Pure Python implementations are slow (~0.2ms per operation)
   - C-optimized libraries (PyCryptodome) are 30-100x faster
   - For production use, always use optimized libraries

4. MODES OF OPERATION:
   - ECB: Simple but not recommended (reveals patterns)
   - EAX: Authenticated encryption with random nonce (recommended)
   - Each mode has different security properties

5. KEY CRACKING:
   - 8-bit key: ~0.05 seconds to crack
   - 16-bit key: ~13 seconds to crack
   - 24-bit key: ~1 hour to crack
   - 128-bit key: Computationally infeasible (10^30+ years)

6. BEST PRACTICES:
   - Use well-established algorithms (AES, not custom ciphers)
   - Use authenticated encryption modes (EAX, GCM, CCM)
   - Use production-grade libraries (PyCryptodome, cryptography)
   - Never roll your own encryption unless you're an expert
   - Keep keys secure and use strong key derivation functions
""")

print("="*70)

KEY TAKEAWAYS: AES ENCRYPTION AND DECRYPTION

1. ENCRYPTION BASICS:
   - AES operates on 16-byte blocks
   - Plaintext must be padded to multiple of 16 bytes
   - Same plaintext + same key = same ciphertext (in ECB mode)

2. KEY SECURITY:
   - Without the correct key, decryption produces garbage
   - Small key differences produce completely different ciphertexts
   - This is the "avalanche effect" - essential for security

3. PERFORMANCE:
   - Pure Python implementations are slow (~0.2ms per operation)
   - C-optimized libraries (PyCryptodome) are 30-100x faster
   - For production use, always use optimized libraries

4. MODES OF OPERATION:
   - ECB: Simple but not recommended (reveals patterns)
   - EAX: Authenticated encryption with random nonce (recommended)
   - Each mode has different security properties

5. KEY CRACKING:
   - 8-bit key: ~0.05 seconds to crack
   - 16-bit key: ~13 seconds to crack
   - 24-bit key: ~1 hour to crack
   - 128-bit key: Computationally infeasible (10^3